In [1]:
import json
import pandas as pd
import numpy as np 
import requests
import seaborn as sns
from datetime import datetime
from requests.auth import HTTPDigestAuth
from matplotlib import pyplot as plt

/Users/marantz/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
# PUB_KEY="wmhpimjc"
# PRIVATE_KEY="00634500-6801-4399-8463-e94c09df6521"
# GROUP_ID="5cc99cd879358e1a071da4a5"\
# # curl -i -u "{PUBLIC-KEY}:{PRIVATE-KEY}"
# # --digest "https://cloud.mongodb.com/api/atlas/v1.0/groups/{GROUP-ID}/processes/
# # atlas-39m1v8-shard-00-01.pmezp.mongodb.net:27017
# # measurements?granularity=PT1M&period=2H&pretty=true"
# RESOURCE_ID="atlas-rvoilc-shard-00-02.pmezp.mongodb.net:27017"

# BASE_URL="https://cloud.mongodb.com/api/atlas/v1.0/groups/"
# REQUESTS_URL=BASE_URL + GROUP_ID + "/processes/"+ RESOURCE_ID +"/measurements?granularity=PT1M&period=PT1H"
# REQUESTS_URL

In [ ]:
# response=requests.get(REQUESTS_URL, auth=HTTPDigestAuth(PUB_KEY, PRIVATE_KEY))

In [ ]:
# response.status_code 

In [ ]:
# response.text

In [ ]:
#dict = json.load(json_file)
dict={}

#file_path='/Users/marantz/Desktop/WORKs/KBS/mediafactory/metrics/'
#file_name='status_mediafactory-prod-shard-00-01.edwaq.mongodb.net_27017_20211022003529.json'
#file_path='/Users/marantz/Sources/ZX_Temp/'
#file_name='status_mediafactory-prod-shard-00-01.edwaq.mongodb.net:27017_20211022141724.json'
#
file_path='/Users/marantz/Desktop/WORKs/Dreamus/METRICS/mongodb_status_1022/'
file_name='status_MMATEtx-mgod-02_PT10S_PT24H_20211021172342.json'

full_path=file_path+file_name

with open(full_path) as json_file:
    dict = json.load(json_file)

In [ ]:
results=dict['measurements']

In [ ]:
#results

In [ ]:
df=pd.DataFrame(results)
df

In [ ]:
names=df['name'].unique()
names

In [ ]:
units=df['units'].unique()
units

In [ ]:
df.keys()

In [ ]:
WORKLOAD_LISTS=[
#'OP_EXECUTION_TIME_READS','OP_EXECUTION_TIME_WRITES',
#'NETWORK_BYTES_IN','NETWORK_BYTES_OUT'
#'PROCESS_CPU_USER','PROCESS_CPU_KERNEL',
'SYSTEM_CPU_USER','SYSTEM_CPU_KERNEL',
]

def getDataPoints(dataPoints):
    timestamps=[]
    values=[]
    for dataPoint in dataPoints:
        #if dataPoint['value'] != None:
        timestamps.append(datetime.strptime(dataPoint['timestamp'], "%Y-%m-%dT%H:%M:%S%z"))
        values.append(dataPoint['value'])
        # Bytes
        #values.append(dataPoint['value']/1024)
    return values,timestamps

data_names=[]
data_timestamp=[]
data_values=[]
data_count=0

for result in results:
    #if 'CPU' in result['name'] :
    if result['name'] in WORKLOAD_LISTS :
        values,timestamps = getDataPoints(result['dataPoints'])
        data_names.append( result['name'] )
        #data_timestamp.append( timestamps )
        data_timestamp = timestamps
        data_values.append( values )
        data_count+=1

print('data_count  : ' + str(data_count))
metric_df=pd.DataFrame(list(map(list, zip(*data_values))),index=data_timestamp,columns=data_names)
metric_df.index.name = 'time'
metric_df


# Workload Basic Operations

In [ ]:
#metric_df.plot()
plt.figure(figsize=(20,10))
plt.title('Workload Basic Operations')
sns.lineplot(data=metric_df)

In [ ]:
start_time=pd.Timestamp("2021-10-21 01:55:00+00:00")
end_time=pd.Timestamp("2021-10-21 02:39:00+00:00")

#metric_df_new=metric_df.between_time('3:00','4:00',True,False,0)
metric_df_new=metric_df[(metric_df.index >= start_time) & (metric_df.index < end_time)]

plt.figure(figsize=(20,10))
plt.title('Workload Basic Operations')
sns.lineplot(data=metric_df_new)

In [ ]:
metric_df_new=metric_df
metric_df_new['dummy'] = 1

print_df=metric_df_new.groupby(lambda x: True).agg(['min','mean','max'])
print_df.drop('dummy', inplace=True, axis=1)
#print_df=print_df.transpose()
print_df

In [ ]:
with pd.ExcelWriter('measurements.xlsx') as writer:
    print_df.to_excel(writer, sheet_name="net_in_out")

In [ ]:
metric_df.head(1).index

In [ ]:
metric_df.tail(1).index

In [ ]:
metric_df_new.head(1).index

In [ ]:
metric_df_new.tail(1).index